<a href="https://colab.research.google.com/github/jmins33/OTT_project/blob/master/Code/Watcha/Watcha_Blog_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [ ]:
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm

In [ ]:
#query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')
# query_txt2 = input('2.제외할 첫번째 키워드를 입력하세요: ')
# query_txt3 = input('3.제외할 두번째 키워드를 입력하세요: '|)

#start_date = input('4.조회를 시작할 날짜를 입력하세요(예:2017-01-01) :')
#end_date = input('5.조회를 종료할 날짜를 입력하세요(예:2017-12-31): ')

In [ ]:
query_txt="왓챠"
start_date="20190101"
end_date="20190228"

In [ ]:
#Step 1. 크롬 웹브라우저 실행
path = "C:/Users/user/Untitled Folder/chromedriver.exe"

driver = webdriver.Chrome(path)
# 사이트 주소는 네이버
driver.get('http://www.naver.com')
time.sleep(2)

In [ ]:
#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 '이재용'
element.submit()


In [ ]:
#Step 3. "블로그" 카테고리 선택
driver.find_element_by_link_text("블로그").click( )    # .click() 괄호 안을 눌러라는 뜻

In [ ]:
#Step 4. 오른쪽의 검색 옵션 버튼 클릭
driver.find_element_by_id("_search_option_btn").click( )

In [ ]:
#Step 5. 정렬 : "관련도순" 
# 개발자 도구에서 정렬 버튼의 id 가 보이지 않습니다.
# 이럴 경우 쉽게 사용할 수 있는 방법이 바로 xpath 를 이용하는 방법입니다.
# xpath는 개발자 도구에서 해당 메뉴 부분을 마우스 오른쪽 버튼을 누르고 
# copy -> copy xpath 를 선택하면 됩니다
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click( )  # 정렬 버튼의 xpath 클릭
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click( ) # 관련도순 xpath

In [ ]:
#Step 6. 날짜 입력
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[2]/a""").click()
time.sleep(2)

In [ ]:
# 이 부분이 아주 중요합니다.
# 날짜 부분에 날짜를 입력할 때 입력 속도가 너무 빠를 경우 날짜가 입력이 되다가 
# 오타가 나오는 경우가 많습니다.
# 그래서 날짜를 입력할 때 for 반복문을 사용해서 1 글자씩 입력하도록 코딩했습니다.

# 시작 날짜 입력하기
s_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""")     
driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""").click()
s_date.clear( )  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
time.sleep(1)
# 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력하는 부분입니다.
for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.1)
    
# 종료 날짜 입력하기
e_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""")
driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""").click()
e_date.clear()
time.sleep(1)

for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.1)
    
# Step7. 날짜 입력 "적용하기" 버튼을 클릭 합니다.  
driver.find_element_by_class_name("tx").click()
time.sleep(3)

# Step 8. 상세 검색버튼을 클릭 후 제외할 단어들을 설정합니다.
#ele2 = driver.find_element_by_id("메탈슬러그")
#ele2.send_keys(query_txt2)
#ele2.send_keys(',')
#ele2.send_keys(query_txt3)
#driver.find_element_by_css_selector(".btn_ft.ty_green._search").click( )

## 글 url 크롤링 시작

In [ ]:
url_list = []
title_list = []

# ★몇개의 페이지를 크롤링할지 선택
total_page = 50
for i in tqdm(range(0, total_page)):  # 페이지 번호
    i = i*10 + 1
    url = "https://search.naver.com/search.naver\
?date_from={0}&date_option=8&date_to={1}\
&dup_remove=1&nso=p%3Afrom{2}to{3}post_blogurl=\
&post_blogurl_without=&query={4}&sm=tab_pge&srchby=all&st=sim&where=post&start={5}".format(start_date,end_date,start_date,end_date,query_txt, i)
    driver.get(url)
    time.sleep(0.5)
    
    # URL 크롤링 시작
    titles = "a.sh_blog_title._sp_each_url._sp_each_title"
    article_raw = driver.find_elements_by_css_selector(titles)
#     article_raw

    # url 크롤링 시작    
    for article in article_raw:
        url = article.get_attribute('href')   
        url_list.append(url)
    
    # 제목 크롤링 시작    
    for article in article_raw:
        title = article.get_attribute('title')   
        title_list.append(title)
    
        print(title)
    
#print('url갯수: ', len(url_list))
#print('url갯수: ', len(title_list))

df = pd.DataFrame({'url':url_list, 'title':title_list})

# 저장하기
df.to_excel("C:/Users/user/Desktop/project/blog_url_watcha_20190101.xlsx")

삼성 스마트티비에서 왓챠플레이 하기/삼성디지털티비 왓차플레이
왓챠, 영화 1100편 돌파! 마시우의 취향을 알아보자
왓챠플레이 크리미널 마인드 오역 모음 (시즌4-6)
왓챠 활동 성적표.. 그리고 콘텐츠 프로토콜 토큰(CPT) 서비스 두번째 이야기
넷플릭스 해지 간단하다, 한달무료 이용 후 재가입했지만 내 스타일은 왓챠플레이 인걸로~
넷플릭스 사용 후기:: 한 달 무료 체험으로 우선 쉽게 즐겨봐~! (feat. 왓챠플레이와 비교)
왓챠 영화추천, 재밌게 본 영화 추천해요 :)
왓챠의 콘텐츠 프로토콜 프로젝트!
왓챠플레이 정기결제/구독취소 가격•_•아이폰기준입니다 ~ 넥플릭스 비교~첫달무료로 카드캡쳐 체리 보는중 ㅋㅋㅋㅋ
[홋카이도 유학 D+322] 세이코마트 포인트카드 & 왓챠플레이
다양한 콘텐츠의 영상을 접하고 싶다면 '왓챠플레이'
왓챠 영화추천 #3
겨울방학엔 정주행! 책임질 왓챠플레이 & 넷플릭스 영화, 드라마 추천(feat. 좀비)
[기록]넷플릭스 & 왓챠 고인물의 드라마, 영화추천 - 2탄 왓챠 플레이
빅리틀라이즈 시즌1 리뷰 (스포포함) 왓챠추천 미드추천
[왓챠플레이] 시간 순삭당한 일드 추천! 언내추럴(완결)
왓챠플레이 / '그렇게 아버지가 된다' 별점 5개
왓챠 VS 넷플릭스, 둘 다 써봤어요! 상세 비교해보기
190217 왓챠, 제발
왓챠 플레이 구독 해지 ! 아이폰 구독 해지 하는 방법
[후기/전자기기] 미니빔+크롬캐스트로 왓챠플레이, 넷플릭스 재생하기
왓챠 CPT(콘텐츠프로토콜토큰)를 업비트에서 매도, 출금하는 방법
2019년 1월 첫째주 드라마/영화 후기+추천 (왓챠)
왓챠플레이 기기초기화 하는 방법!!
왓챠플레이 영화 추천!! 로맨틱코미디 <우리 사랑일까요?>
10. 왓챠플레이를 시작했다 - SATC 베스트 에피를 찾아서
[왓챠, 네이버tv] 웹드라마 먹는 존재
왓챠플레이 한달 무료 이용권 사용 해지 후기
인기 해외 드라마 시리즈 추천 (왓챠 플레이)
[왓챠] 영화 / 독서 취향분석 및 추천
[정보] 영화어플 왓챠플레이 콘텐츠프로토콜 C

[왓챠플레이] 숀더쉽
왓챠 도서 평점2
[왓챠플레이] 어린 왕자
[왓챠플레이] 렛미인
[왓챠플레이] 별의 목소리
스포트라이트 (왓챠플레이 )
라따뚜이 움짤 간단 줄거리 소개          #왓챠 #추천
[왓챠플레이/미드/시트콤] 실리콘밸리 Silicon Valley 정주행 후기
[공유] [조선일보] 설 연휴에 몰아보면 ‘타임워프’…넷플릭스·왓챠 추천 콘텐츠
왓챠 도서 평점 6 인생, 딸에 대하여
[왓챠플레이] 극장판 위 베어 베어스 : 곰 브라더스
*2018년 왓챠 성적표 성적통지 표(영화/드라마/도서)
한달 동안 왓챠에서 본 영화/드라마
넷플/왓챠 미드 기록
왓챠 700편 돌파 / 영화 <극한 직업>
[공유] ⟪블랙⟫ 드라마 | 인생, 희망적인 | 인도 | 2시간 4분 #왓챠플레이  블랙 (2005) Black is a very touching movie.  The movie is about Michelle McNally, who has never seen and heard of, is a story of growing up with teacher Saha. A memorable line in this movie -Life is an icecream , therefore let’
왓챠 킬링타임용 영화 추천! Before I Fall(7번째 내가 죽던 날) , 스포x
코코 (2018. 픽사. 개인 짤막 평, 왓챠 평점)
왓챠 도서 평점 3 구해줘, 나는 까칠하게 살기로 했다, 안녕 주정뱅이
와 ••• + 왓챠
[왓챠,영화] 소공녀
2019 왓챠데미 시상식! 나만의 TOP5는?
왓챠 도서 평점 4 소설가의 일, 스물아홉 생일~, 백설공주에게 죽음을, 파리의 아파트
힐링이 필요할 때 보기 좋은 영화 추천 (넷플릭스, 왓챠플레이)
어디까지나 주관적인 일드 추천 "왓챠편"
181223 / 왓챠데이
[영화] 설연휴에 왓챠플레이에서 본 영화들 짧은 감상 (코렐라인, 라이브 플래쉬, 좀비랜드)
왓챠에서
[왓챠플레이 일드 추천]
[스포일러 리뷰] 월-E (WALL-E, 2

In [ ]:
import sys
import os
import pandas as pd
import numpy as np

In [ ]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("C:/Users/user/Desktop/project/blog_url_watcha_20190101.xlsx")        # 기본 모델

num_list = len(url_load)

print(num_list)
url_load.head()

500


,Unnamed: 0,url,title
0,0,https://blog.naver.com/areum_1202?Redirect=Log...,삼성 스마트티비에서 왓챠플레이 하기/삼성디지털티비 왓차플레이
1,1,https://blog.naver.com/luckymasiwoo?Redirect=L...,"왓챠, 영화 1100편 돌파! 마시우의 취향을 알아보자"
2,2,https://whyjaylee.blog.me/221452905081,왓챠플레이 크리미널 마인드 오역 모음 (시즌4-6)
3,3,https://blog.naver.com/sowon007?Redirect=Log&l...,왓챠 활동 성적표.. 그리고 콘텐츠 프로토콜 토큰(CPT) 서비스 두번째 이야기
4,4,https://blog.naver.com/anboday?Redirect=Log&lo...,"넷플릭스 해지 간단하다, 한달무료 이용 후 재가입했지만 내 스타일은 왓챠플레이 인걸로~"


In [ ]:
dict = {}  # 전체 크롤링 데이터를 담을 그릇

# ★수집할 글 갯수
number = 500
for i in tqdm(range(0, number)): 
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to_frame('mainFrame')

        target_info = {}

        # 제목 크롤링 시작
        overlays = ".se-fs-.se-ff-"                                 
        tit = driver.find_element_by_css_selector(overlays)         # title
        title = tit.text
        title

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nick
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # date
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)         # date

        content_list = []
        for content in contents:
            content_list.append(content.text)

        content_str = ' '.join(content_list)

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        #print(i, title)

        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("watcha_20190101_20190228.xlsx")
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)


# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')

# 저장하기
result_df.to_excel("C:/Users/user/Desktop/project/watcha_20190101_20190228.xlsx")

C:\Users\user\.conda\envs\media\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use driver.switch_to.frame instead
  from ipykernel import kernelapp as app
